<a href="https://colab.research.google.com/github/linhvien/Data-Scraping/blob/main/Yahoo_Finance_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extract financial data of stocks from Yahoo Finance 

#Import Libraries

In [ ]:
from bs4 import BeautifulSoup
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Financial data

In [ ]:
# Define Url and establish connection
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [ ]:
def getannualisdata(stock):
  # Use requests to retrieve data from a given URL
  response = requests.get(url_financials.format(stock,stock),headers=headers)
  soup = BeautifulSoup(response.text,'html.parser')

  # Use reg expression \s for non-breaking spaces
  pattern = re.compile(r'\s--\sData\s--\s')
  script_data = soup.find('script',text=pattern).contents[0]

  #Extracting and parsing html and json data
  start = script_data.find('context')-2
  json_data = json.loads(script_data[start:-12])

  annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
  annual_is_stmts = []
  for s in annual_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)
  return annual_is_stmts

In [ ]:
df = pd.DataFrame(getannualisdata('AMZN'))
df['endDate'] = df['endDate'].astype('datetime64[s]')
df.set_index('endDate', inplace=True)
df

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
endDate,,,,,,,,,,,,,,,,
2020-12-31,42740000000,24194000000,21331000000,87193000000,152757000000,22899000000,22899000000,-75000000,-1647000000,2863000000,386064000000,363165000000,233307000000,1295000000,21331000000,21331000000
2019-12-31,35931000000,13962000000,11588000000,64313000000,114986000000,14404000000,14404000000,338000000,-1600000000,2374000000,280522000000,266118000000,165536000000,-442000000,11588000000,11588000000
2018-12-31,28837000000,11270000000,10073000000,52177000000,93731000000,12421000000,12421000000,296000000,-1417000000,1197000000,232887000000,220466000000,139156000000,-1151000000,10073000000,10073000000
2017-12-31,22620000000,3802000000,3033000000,38992000000,65932000000,4106000000,4106000000,214000000,-848000000,769000000,177866000000,173760000000,111934000000,-304000000,3033000000,3033000000


In [ ]:
portfolio = ['AMZN','AAPL','TSLA']
dfs = []
for i in portfolio:
  dfs.append(pd.DataFrame(getannualisdata(i)))
df = pd.concat(dfs, axis=0, ignore_index=True)
date = df['endDate'].astype('datetime64[s]').tolist()
df['endDate'] = df['endDate'].astype('datetime64[s]')
df.set_index('endDate', inplace=True)

index = pd.MultiIndex.from_product([portfolio,date])

df

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,minorityInterest
endDate,,,,,,,,,,,,,,,,,
2020-12-31,42740000000,24194000000,21331000000,87193000000,152757000000,22899000000,22899000000,-75000000.0,-1647000000,2863000000,386064000000,363165000000,233307000000,1295000000,21331000000,21331000000,NaN
2019-12-31,35931000000,13962000000,11588000000,64313000000,114986000000,14404000000,14404000000,338000000.0,-1600000000,2374000000,280522000000,266118000000,165536000000,-442000000,11588000000,11588000000,NaN
2018-12-31,28837000000,11270000000,10073000000,52177000000,93731000000,12421000000,12421000000,296000000.0,-1417000000,1197000000,232887000000,220466000000,139156000000,-1151000000,10073000000,10073000000,NaN
2017-12-31,22620000000,3802000000,3033000000,38992000000,65932000000,4106000000,4106000000,214000000.0,-848000000,769000000,177866000000,173760000000,111934000000,-304000000,3033000000,3033000000,NaN
2020-09-26,18752000000,67091000000,57411000000,19916000000,104956000000,66288000000,66288000000,NaN,-2873000000,9680000000,274515000000,208227000000,169559000000,803000000,57411000000,57411000000,NaN
2019-09-28,16217000000,65737000000,55256000000,18245000000,98392000000,63930000000,63930000000,NaN,-3576000000,10481000000,260174000000,196244000000,161782000000,1807000000,55256000000,55256000000,NaN
2018-09-29,14236000000,72903000000,59531000000,16705000000,101839000000,70898000000,70898000000,NaN,-3240000000,13372000000,265595000000,194697000000,163756000000,2005000000,59531000000,59531000000,NaN
2017-09-30,11581000000,64089000000,48351000000,15261000000,88186000000,61344000000,61344000000,NaN,-2323000000,15738000000,229234000000,167890000000,141048000000,2745000000,48351000000,48351000000,NaN
2020-12-31,1491000000,1154000000,721000000,3188000000,6630000000,1951000000,1951000000,NaN,-784000000,292000000,31536000000,29585000000,24906000000,-797000000,862000000,690000000,1.454000e+09


In [ ]:
mystocks = ['AMZN','AAPL','TSLA']
stockdata = []
for i in mystocks:
  stockdata.append(getannualisdata(i))
  print('Getting:',i)

with open('stockdata.json','w') as f:
  json.dump(stockdata, f)

Getting: AMZN
Getting: AAPL
Getting: TSLA


#Explain details how to extract and parse html and json data

In [ ]:
stock = 'AMZN'
response = requests.get(url_financials.format(stock,stock),headers=headers)
soup = BeautifulSoup(response.text,'html.parser')

Notice that there is a lot of javascript and not a lot of html to work with. Also notice that json formatted text string is embedded in the code. There is a javascript function commentted with "--Data--", and this function is located inside of a generic "script" tag. 

In [ ]:
# Use reg expression \s for non-breaking spaces
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script',text=pattern).contents[0]

In [ ]:
script_data[:500]

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1632296362274;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

In [ ]:
#the end
script_data[-500:]

'put":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-adfeedback":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};\n}(this));\n'

In [ ]:
# find the starting position of the json string
start = script_data.find('context')-2

# There are 12 characters from the end, and then slice the json string
json_data = json.loads(script_data[start:-12])

In [ ]:
json_data['context'].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [ ]:
json_data['context']['dispatcher']['stores'].keys()

dict_keys(['PageStore', 'MRTStore', 'RouteStore', 'I13nStore', 'PageTransitionStore', 'VideoPlayerStore', 'QuoteAutoCompleteStore', 'FlyoutStore', 'NavrailStore', 'StreamDataStore', 'QuoteSummaryStore', 'FinanceConfigStore', 'LangStore', 'BeaconStore', 'AdStore', 'VideoStore', 'ComponentConfigStore', 'CrumbStore', 'CompositeStore', 'StreamStore', 'UserStore', 'ProfileStore', 'QuotePageStore', 'NavServiceStore', 'FinancialTemplateStore', 'QuoteTimeSeriesStore', 'MarketTimeStore', 'MarketSummaryStore', 'UHAccountSwitchStore', 'RecommendationStore', 'MobileHeaderStore'])

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [ ]:
# income statement
annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

# cash flow statement
annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

# balance sheet
annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

In [ ]:
annual_is_stmts = []

# consolidate annual
for s in annual_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['longFmt']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)

#Statistics data

In [ ]:
stock = 'AAPL'
r = requests.get(url_stats.format(stock,stock),headers=headers)
statistics = pd.read_html(r.text)

valuation_Measures = statistics[0]
stock_Price_History = statistics[1]
   
share_Statistics = statistics[2]

dividend_Info = statistics[3]
profitability_Info = statistics[5]
management_Efectiveness = statistics[6]

income_Statement = statistics[7]
balance_Sheet = statistics[8]
cash_Flow = statistics[9]

income_Statement

,0,1
0,Revenue (ttm),347.16B
1,Revenue Per Share (ttm),20.61
2,Quarterly Revenue Growth (yoy),36.40%
3,Gross Profit (ttm),104.96B
4,EBITDA,110.93B
5,Net Income Avi to Common (ttm),86.8B
6,Diluted EPS (ttm),5.11
7,Quarterly Earnings Growth (yoy),93.20%


#Historical data

In [ ]:
class historicaldata:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    def __init__(self, stock, days_back=7):
        self.stock = stock
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.stock), headers=headers, timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.stock, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url,headers=headers)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [ ]:
df = historicaldata('AAPL', days_back=3000).get_quote()
df.set_index('Date', inplace=True)
df[['Adj Close']].tail()

,Adj Close
Date,
2021-09-15,149.029999
2021-09-16,148.789993
2021-09-17,146.059998
2021-09-20,142.940002
2021-09-21,143.429993
